In [ ]:
import numpy as np
import cv2
import copy
import os, matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
from MyUtils import transforms, utils, engine, train as transforms, utils, engine, train
from MyUtils.utils import collate_fn
from MyUtils.engine import train_one_epoch, evaluate
from MyUtils.plot_statistic import plot_stats
from MyUtils.visulaize import visualize
import MyUtils.Dataset

In [ ]:

#from pprint import pprint
#import json

#with open(os.path.join('C:/Users/User/Petr/Net_2/NEW_DATASET/annotation.json'), 'r') as file:
#    annotations = json.load(file)

#pprint(annotations[0]['annotations'][0]['result'])

In [ ]:
#import shutil

#path = 'C:/Users/User/Petr/Net_2/NEW_DATASET/9c_dataset/'

#images = [file for file in sorted(os.listdir(os.path.join(path, "images")))]
#annotations = [file for file in sorted(os.listdir(os.path.join(path, "annotations"))) if not file=='classes.txt']
#count = 0
#for annotation in annotations:
#    with open(path + 'annotations/' + annotation, 'r') as file:
#        lines = file.readlines()
#        num_corners = len([line for line in lines if line.startswith('0')])
#        if 2 <= num_corners <= 5:
#            shutil.copyfile(path + 'annotations/' + annotation, f'C:/Users/User/Petr/Net_2/NEW_DATASET/5c_dataset/annotations/{annotation}')
#            shutil.copyfile(path + 'images/' + annotation[:-4] + '.jpg', f'C:/Users/User/Petr/Net_2/NEW_DATASET/5c_dataset/images/{annotation[:-4]}.jpg')

In [ ]:
#temp_folder = './test_dataset/train/temp/'

#imgs = os.listdir(temp_folder)

#for num, img in enumerate(imgs):
#    os.rename(os.path.join(temp_folder, img), os.path.join(temp_folder, f'dataset_{num + 1}.jpg'))

In [ ]:
def train_transform():
    return A.Compose([
        A.Sequential([
            A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            A.RGBShift(r_shift_limit=30, g_shift_limit=30, b_shift_limit=20, always_apply=False, p=0.5),
            A.PixelDropout(dropout_prob=0.005, per_channel=False, drop_value=0, mask_drop_value=None, always_apply=False, p=0.5),
            A.Affine(scale=(0.8, 1.0), translate_percent=(-0.1, 0.1), translate_px=None, rotate=0, shear=None, interpolation=1, mask_interpolation=0, cval=0, cval_mask=0, mode=0, fit_output=False, keep_ratio=True, rotate_method='largest_box', always_apply=False, p=0.5),
            #A.RandomScale(scale_limit=[0.1, 0.5], interpolation=1, always_apply=False, p=0.5),
            #A.CropAndPad(px=None, percent=-0.2, pad_mode='BORDER_CONSTANT', pad_cval=100, pad_cval_mask=0, keep_size=False, sample_independently=True, interpolation=1, always_apply=False, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=0.1, brightness_by_max=True, always_apply=False, p=0.5), # Random change of brightness & contrast
            #A.ToGray(p=1),
            #A.InvertImg(p=0.5),
            #A.ToGray(p=1)
            #A.Sharpen(alpha=(0.5, 0.5), p=1)
        ])
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

def eval_transform():
    return A.Compose([
        A.Sequential([
            #A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            #A.RandomBrightnessContrast(brightness_limit=(-0.5, 0.0), contrast_limit=0.0, brightness_by_max=True, always_apply=False, p=1.0), # Random change of brightness & contrast
            #A.InvertImg(p=1),
            #A.ToGray(p=1)
            #A.Sharpen(alpha=(0.5, 0.5), p=1)
        ], p=1)
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

def test_transform():
    return A.Compose([
        A.Sequential([
            #A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            #A.RandomBrightnessContrast(brightness_limit=(-0.5, 0.0), contrast_limit=0.0, brightness_by_max=True, always_apply=False, p=1.0), # Random change of brightness & contrast
            #A.InvertImg(p=1),
            #A.ToGray(p=1)
            #A.Sharpen(alpha=(0.1, 0.9), p=1)
        ], p=1)
    ])

In [ ]:
path = 'C:/Users/User/Petr/Net_2/NEW_DATASET'

KEYPOINTS_FOLDER = path
KEYPOINTS_FOLDER_TRAIN = path

dataset = MyUtils.Dataset.NCornerDataset(KEYPOINTS_FOLDER, transform=None, demo=False, corners=[2, 3])

dataset.explore

In [ ]:
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

iterator = iter(data_loader)
batch = next(iterator)

image = (batch[0][0].permute(1,2,0).numpy() * 255).astype(np.uint8)
bboxes = batch[1][0]['boxes'].detach().cpu().numpy().astype(np.int32).tolist()

idx = torch.IntTensor.item(batch[1][0]['image_id'])

print(dataset.get_image(dataset.annotations[idx]))

keypoints = []
for kps in batch[1][0]['keypoints'].detach().cpu().numpy().astype(np.int32).tolist():
    keypoints.append([kp for kp in kps])

print(keypoints)
visualize(image, bboxes, keypoints)

In [ ]:
dataset = MyUtils.Dataset.NCornerDataset(KEYPOINTS_FOLDER_TRAIN, transform=train_transform(), demo=True, corners=[2, 3])
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

iterator = iter(data_loader)
batch = next(iterator)

image = (batch[0][0].permute(1,2,0).numpy() * 255).astype(np.uint8)
bboxes = batch[1][0]['boxes'].detach().cpu().numpy().astype(np.int32).tolist()
idx = torch.IntTensor.item(batch[1][0]['image_id'])

print(dataset.get_image(dataset.annotations[idx]))

keypoints = []
for kps in batch[1][0]['keypoints'].detach().cpu().numpy().astype(np.int32).tolist():
    keypoints.append([kp for kp in kps])

image_original = (batch[2][0].permute(1,2,0).numpy() * 255).astype(np.uint8)
bboxes_original = batch[3][0]['boxes'].detach().cpu().numpy().astype(np.int32).tolist()

keypoints_original = []
for kps in batch[3][0]['keypoints'].detach().cpu().numpy().astype(np.int32).tolist():
    keypoints_original.append([kp for kp in kps])

visualize(image, bboxes, keypoints, image_original, bboxes_original, keypoints_original)
print(bboxes)
print(keypoints)

In [ ]:
def get_model(num_keypoints, weights_path=None):
   
    
    anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 2.0, 3.0, 4.0))
    model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=True,
                                                                   pretrained_backbone=True,
                                                                   num_keypoints=num_keypoints,
                                                                   num_classes = 2, # Background is the first class, object is the second class
                                                                   rpn_anchor_generator=anchor_generator)

    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)        
        
    return model


def get_model_V2(num_keypoints, weights_path=None):
    from torchvision.ops import MultiScaleRoIAlign
    from torchvision.models.detection import KeypointRCNN
    
    backbone = torchvision.models.mobilenet_v2(weights='DEFAULT').features
    
    backbone.out_channels = 1280
    
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512, 1024),), aspect_ratios=((0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 2.0, 3.0, 4.0),))
    
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0', '1', '2', '3'], output_size=7, sampling_ratio=2)
    
    
    keypoint_roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0', '1', '2', '3'],  output_size=14, sampling_ratio=2)
    
    model = KeypointRCNN(backbone, num_classes=2, num_keypoints=num_keypoints, rpn_anchor_generator=anchor_generator, box_roi_pool=roi_pooler, keypoint_roi_pool=keypoint_roi_pooler)#, min_size=400, max_size=2000

    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)        
        
    return model

def get_model_V3(num_keypoints, weights_path=None):
    from torchvision.models.detection.keypoint_rcnn import KeypointRCNNPredictor
    from torchvision.models.detection.rpn import AnchorGenerator
    from torchvision.models.detection import keypointrcnn_resnet50_fpn
    # Load a pre-trained model
    model = keypointrcnn_resnet50_fpn(weights='DEFAULT')

# Replace the classifier head with the number of keypoints
    in_features = model.roi_heads.keypoint_predictor.kps_score_lowres.in_channels
    model.roi_heads.keypoint_predictor = KeypointRCNNPredictor(in_channels=in_features, num_keypoints=num_keypoints)

# Set the model's device and data type
    model.name = 'keypointrcnn_resnet50_fpn'
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)        
        
    return model
    

In [ ]:
save_path = 'C:/Users/User/Petr/Net_2/save_model'
log_path = 'C:/Users/User/Petr/Net_2/Metric_log'

for num in range(1):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#device = torch.device('cpu')
    
    KEYPOINTS_FOLDER_TRAIN = path
    KEYPOINTS_FOLDER_EVAL = path
    corners = [2,3]
    POINTS = max(corners)

    dataset_train = MyUtils.Dataset.NCornerDataset(KEYPOINTS_FOLDER_TRAIN, transform=train_transform(), demo=False, corners=corners)
    dataset_test = MyUtils.Dataset.NCornerDataset(KEYPOINTS_FOLDER_EVAL, transform=eval_transform(), demo=False, corners=corners)

    indices = torch.randperm(len(dataset_train)).tolist()
    thirty_pc = int(len(dataset_train) * 0.15)
    dataset_train = torch.utils.data.Subset(dataset_train, indices[:-thirty_pc])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-thirty_pc:])




    data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
    data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

    model = get_model_V3(num_keypoints = POINTS)#, weights_path='./save_model/weights_rebuild_model_2.pth'


    model.to(device)
    
    #for p in model.parameters():
    #    p.requires_grad = False
    #for p in model.roi_heads.keypoint_head.parameters():
    #    p.requires_grad = True
    #for p in model.roi_heads.keypoint_predictor.parameters():
    #    p.requires_grad = True

    params = [p for p in model.parameters() if p.requires_grad]
    #params1 = [p for p in model.roi_heads.keypoint_head.parameters() if p.requires_grad]
    
    optimizer = torch.optim.SGD(params, lr=2e-4, momentum=0.90)#, weight_decay=0.0001
    
    #optimizer = torch.optim.SGD([{'params': params1},
    #                             {'params': model.roi_heads.keypoint_predictor.parameters(), 'lr': .001},], lr=0.001, momentum=0.90)#, weight_decay=0.0001
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.9)
    num_epochs = 1001

    kps_stats = []
    bbox_stats = []

    loss_bb = []
    loss_kp = []
    loss = []

    e_done = 0

    for epoch in range(e_done, num_epochs):
        logger = train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=20)
        lr_scheduler.step()
        evaluator = evaluate(model, data_loader_test, device)
        
        kps_stats.append(evaluator.coco_eval['keypoints'].stats[:6])
        bbox_stats.append(evaluator.coco_eval['bbox'].stats[:6])
    
        loss_bb.append(logger.meters['loss_box_reg'].global_avg)
        loss_kp.append(logger.meters['loss_keypoint'].global_avg)
        loss.append(logger.meters['loss'].global_avg)
        
        if epoch % 100 == 0 and epoch > 0:
            torch.save(model.state_dict(), f'{save_path}/LS/2_3c/weights_{epoch}.pth')
            plot_stats(epoch+1 - e_done, bbox_stats, kps_stats, loss_bb, loss_kp, loss, num=epoch, log_path=log_path)#, num=num
            
    plot_stats(num_epochs - e_done, bbox_stats, kps_stats, loss_bb, loss_kp, loss, num=(epoch+1), show=True, log_path=log_path)
#print(type(meters[0]['loss_keypoint']))

# Save model weights after training
#torch.save(model.state_dict(), './save_model/weights_2.pth')
    #torch.save(model.state_dict(), f'{save_path}/2c/weights_2c_re_1000.pth')

In [ ]:
KEYPOINTS_FOLDER = './'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_test = get_model_V2(num_keypoints = 2, weights_path='./save_model/weights_2c_2200.pth')
model_test.to(device);

In [ ]:
test_path = 'C:/Users/User/Petr/Net_2/check_model/Rod/images/'

for image in tqdm(os.listdir(test_path)):
    img = cv2.imdecode(np.fromfile(os.path.join(test_path, image), dtype=np.uint8), cv2.IMREAD_UNCHANGED) #cv2.imread(os.path.join(test_path, image))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img_orig = cv2.imdecode(np.fromfile(os.path.join(test_path, image), dtype=np.uint8), cv2.IMREAD_UNCHANGED)
    img_orig = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    
    img = F.to_tensor(img)
    type(img)
    img = img.to(device)
    with torch.no_grad():
        model_test.to(device)
        model_test.eval()
        out = model_test([img,])
        out = out[0]
    #img = (img[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
    scores = out['scores'].detach().cpu().numpy()
    kps_scores = out['keypoints_scores'].detach().cpu().numpy()

    high_scores_idxs = np.where(scores > 0.75)[0].tolist() # Indexes of boxes with scores > 0.7
    post_nms_idxs = torchvision.ops.nms(out['boxes'][high_scores_idxs], out['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)

# Below, in output[0]['keypoints'][high_scores_idxs][post_nms_idxs] and output[0]['boxes'][high_scores_idxs][post_nms_idxs]
# Firstly, we choose only those objects, which have score above predefined threshold. This is done with choosing elements with [high_scores_idxs] indexes
# Secondly, we choose only those objects, which are left after NMS is applied. This is done with choosing elements with [post_nms_idxs] indexes
#    print(out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0])
    if not len(out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()):
        good_kps_idxs = [[]]
    else:
        good_kps_idxs = np.where(out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0] > 5.0) # for i in range(len(post_nms_idxs))
#    print(good_kps_idxs[0])
#    print('KPs: ', out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0][good_kps_idxs[0]])
    #kps_scores_filtered = out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0][good_kps_idxs[0]]
#print(kps_scores_filtered)

#kps_filtered =  output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
#out['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
#print(kps_filtered)

    keypoints = []
    for kps in out['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
        keypoints.append([list(map(int, kp)) for kp in kps[good_kps_idxs[0]]])
    scores = kps_scores[0][good_kps_idxs[0]]
    #kps_scores = out['keypoints_scores'][high_scores_idxs][post_nms_idxs][good_kps_idxs[0]].detach().cpu().numpy()[0]
#kps_idxs = np.where(output[0]['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0] > 1.2)[0]
#print(kps_idxs)
#print(np.array(keypoints)[0][kps_idxs])
#keypoints = [np.array(keypoints)[0][kps_idxs].tolist()]
    #print(keypoints)
#keypoints = average_close_points(keypoints)
    #print(keypoints)
    #print(kps_scores[0][good_kps_idxs[0]])
    #print(high_scores_idxs, post_nms_idxs, good_kps_idxs[0])
    #break
    bboxes = []
    for bbox in out['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
        bboxes.append(list(map(int, bbox.tolist())))
    bld_scores = out['scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy().tolist()
    #print(bld_scores)
    #print(len(bboxes), len(bld_scores))
    #print(type(out[0]['boxes'].detach().cpu().numpy().tolist()))
    visualize(img_orig, bboxes, keypoints, label=image, bld_score=bld_scores, kps_scores=scores)